## Test the persitence of the basic translation model

In [10]:
from __future__ import print_function
from __future__ import division

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import io
import pandas as pd
from keras.models import load_model

In [12]:
# Define the path to the training data.  I won't need this later, but it's a good example for
# processing new inputs
data_path = 'data/fra-eng/fra.txt'
num_samples = 10000 
latent_dim = 256 

In [13]:
# Process the data
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with io.open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We need start of sequence character to identify the inputs (this is a tab)
    # Also need an end of sequence character to identify the target (this is \n)
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples: {0}'.format(len(input_texts)))
print('Number of unique input tokens: {0}'.format(num_encoder_tokens))
print('Number of unique output tokens: {0}'.format(num_decoder_tokens))
print('Max sequence length of inputs: {0}'.format(max_encoder_seq_length))
print('Max sequence length of outputs: {0}'.format(max_decoder_seq_length))

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is going to be ahead of decoder_input_data by one timestep.
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data is ahead by one timestep and will not include the start character
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length of inputs: 16
Max sequence length of outputs: 59


In [14]:
# I guess I have to define the model again?
# Process an input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Now we can set up the decoder.  The initial state is then 'encoder_states', the last state of the encoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Now define the model to turn encoder_input_data and decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [17]:
# Reverse-lookup token to decode sequence back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [15]:
model.load_weights('s2s.h5')

In [16]:
model.

In [19]:
# Can I extract the dense intermediate layer?
dense_layer = model.output_layers

TypeError: 'list' object is not callable